In [ ]:
# Building makemore Part 4: Becoming a Backprop Ninja
# https://youtu.be/q8SA3rM6ckI?si=QlPqAFqzpmKGduKW

In [88]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: module://matplotlib_inline.backend_inline


In [89]:
words = open('names.txt', 'r').read().splitlines()

In [90]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)

In [91]:
block_size = 3

def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [92]:
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    # approximate
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [93]:
n_embd = 10 # embd dim
n_hidden = 64 # neurons in hidden layer

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)

In [94]:
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) # not necessary, we have batch norm

In [95]:
# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

In [96]:
# BatchNorm parameters
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

In [97]:
# These initializations are non-standard, because doing all zeros could mask
# an incorrect implementation of the backward pass

In [98]:
parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))

4137


In [99]:
for p in parameters:
    p.requires_grad = True

In [100]:
n = 32 # batch_size
ix = torch.randint(0, Xtr.shape[0], (n,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [101]:
# Forward pass
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)

# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani # deviation from mean, matrix
bndiff2 = bndiff ** 2
bnvar = 1 / (n - 1) * (bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# Cross-entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum ** -1 # if (1.0 / counts_sum), backprop won't be bit exact
probs = counts * counts_sum_inv
logprobs = probs.log()
# going down the rows - range(n) - and always plucking out index in Yb 
loss = -logprobs[range(n), Yb].mean()

In [102]:
# pytorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
          embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.3417, grad_fn=<NegBackward0>)

In [126]:
# Exercise 1: backprop through whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# dlogprobs: derivative of loss in relation to logprobs
dlogprobs = torch.zeros_like(logprobs) # keep shape
dlogprobs[range(n), Yb] = - 1.0 / n # derivative of mean
cmp('logprobs', dlogprobs, logprobs)

# dprobs
dprobs = torch.zeros_like(probs)
dprobs = (1.0 / probs) * dlogprobs # chain rule
cmp('probs', dprobs, probs)

# dcounts_sum_inv
# remember: counts_sum_inv was replicated, and this is the derivative with respect
# to the replicated vectors
# counts * drops is deriv in relation to replicated counts_sum_inv
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True) # keep dim of counts_sum_inv
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

# dcounts
dcounts = counts_sum_inv * dprobs

# dcounts_sum
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)

# dcounts
dcounts += torch.ones_like(counts) * dcounts_sum # broadcasting
cmp('counts', dcounts, counts)

# dnorm_logits
dnorm_logits = (counts) * dcounts # counts = norm_logits.exp()
cmp('dnorm_logits', dnorm_logits, norm_logits)

# dlogits
dlogits = dnorm_logits.clone() # counts = norm_logits.exp()

# dlogit_maxes
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)  # counts = norm_logits.exp()
cmp('dlogit_maxes', dlogit_maxes, logit_maxes)

# dlogits
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
cmp('dlogits', dlogits, logits)

# dh
# shape of dh must be same as shape of h (from matmul)
dh = dlogits @ W2.T
cmp('dh', dh, h)

# dW2
dW2 = h.T @ dlogits
cmp('dW2', dW2, W2)

# db2
db2 = dlogits.sum(0)
cmp('db2', db2, b2)

# dhpreact
dhpreact = (1 - h**2) * dh
cmp('dhpreact', dhpreact, hpreact)

# dhpreact
dhpreact = (1 - h**2) * dh
cmp('dhpreact', dhpreact, hpreact)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
dnorm_logits    | exact: True  | approximate: True  | maxdiff: 0.0
dlogit_maxes    | exact: True  | approximate: True  | maxdiff: 0.0
dlogits         | exact: True  | approximate: True  | maxdiff: 0.0
dh              | exact: True  | approximate: True  | maxdiff: 0.0
dW2             | exact: True  | approximate: True  | maxdiff: 0.0
db2             | exact: True  | approximate: True  | maxdiff: 0.0
dhpreact        | exact: True  | approximate: True  | maxdiff: 0.0
